# RNN

In [146]:
# import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import LSTM
import numpy as np
import torch
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import RNN
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

### Balancing Data

In [147]:
d1 = pd.read_csv("../data/extracted/d1.csv")
d2 = pd.read_csv("../data/extracted/d2.csv")
test = pd.read_csv("../data/extracted/test.csv")

In [148]:
d1_X = d1['txt'].values
d1_Y = d1['label'].values
d2_X = d2['txt'].values
d2_Y = d2['label'].values

In [149]:
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy = 0.5)
# define undersampling strategy
undersample = RandomUnderSampler(sampling_strategy=0.035)

In [150]:
d1_X_r = d1_X.reshape(-1, 1)
d2_X_r = d2_X.reshape(-1, 1)

In [151]:
d1_X_under, d1_Y_under = undersample.fit_resample(d1_X_r, d1_Y)

In [152]:
d2_X_over, d2_Y_over = oversample.fit_resample(d2_X_r, d2_Y)

## Domain 1 dataset

In [153]:
#Splitting dataset into training and testing dataset
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(d1_X_under,d1_Y_under,test_size=0.2,random_state=0)

In [154]:
X_train_r = X_train.reshape(-1,)
X_test_r = X_test.reshape(-1,)

### Tokenisation

In [157]:
t = Tokenizer()
t.fit_on_texts(X_train_r)

In [ ]:
E_train = t.texts_to_sequences(X_train_r)
E_test = t.texts_to_sequences(X_test_r)

In [159]:
max_len = []
sum_len = 0
max_lenth = 0
min_lenght = 10
for i in range(len(E_train)):
    max_len.append(len(E_train[i]))
    sum_len += len(E_train[i])
    if len(E_train[i]) > max_lenth:
        max_lenth = len(E_train[i])
    elif len(E_train[i]) < min_lenght:
        min_lenght = len(E_train[i])
    i += 1

In [ ]:
max_length = 150
padded_train = pad_sequences(E_train, maxlen=max_length, padding='post')
padded_test = pad_sequences(E_test, maxlen=max_length, padding='post')

In [ ]:
words = len(t.word_index) + 1

# define the model
RNN = Sequential()
RNN.add(Embedding(words, 50, input_length=max_length))
RNN.add(SimpleRNN(50, return_sequences=False))
# model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(24)))
RNN.add(Dense(1, activation='sigmoid'))
RNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['TrueNegatives'])

In [162]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
RNN.fit(x=padded_train,y=Y_train,epochs=50,validation_data=(padded_test, Y_test), verbose=1,callbacks=[early_stop])

Epoch 1/50
2588/2588 [==============================] - 54s 20ms/step - loss: 0.1025 - true_negatives: 958.0000 - val_loss: 0.1034 - val_true_negatives: 324.0000
Epoch 2/50
2588/2588 [==============================] - 53s 20ms/step - loss: 0.0974 - true_negatives: 954.0000 - val_loss: 0.1114 - val_true_negatives: 354.0000
Epoch 3/50
2588/2588 [==============================] - 52s 20ms/step - loss: 0.0872 - true_negatives: 1052.0000 - val_loss: 0.1022 - val_true_negatives: 277.0000
Epoch 4/50
2588/2588 [==============================] - 53s 21ms/step - loss: 0.0894 - true_negatives: 1000.0000 - val_loss: 0.1058 - val_true_negatives: 204.0000
Epoch 5/50
2588/2588 [==============================] - 54s 21ms/step - loss: 0.0882 - true_negatives: 1014.0000 - val_loss: 0.1071 - val_true_negatives: 330.0000
Epoch 6/50
2588/2588 [==============================] - 54s 21ms/step - loss: 0.0870 - true_negatives: 1048.0000 - val_loss: 0.1014 - val_true_negatives: 301.0000
Epoch 7/50
2588/2588 [==

In [163]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def c_report(y_true, y_pred):
   print("Classification Report")
   print(classification_report(y_true, y_pred))
   acc_sc = accuracy_score(y_true, y_pred)
   print("Accuracy : "+ str(acc_sc))
   return acc_sc

In [167]:
c_report(Y_test, preds)

Classification Report
              precision    recall  f1-score   support

           0       0.41      0.50      0.45       717
           1       0.98      0.97      0.98     19983

    accuracy                           0.96     20700
   macro avg       0.70      0.74      0.72     20700
weighted avg       0.96      0.96      0.96     20700

Accuracy : 0.9578743961352657


0.9578743961352657

In [164]:
preds = (model.predict(padded_test) > 0.5).astype("int32")

647/647 [==============================] - 3s 5ms/step


## Test

In [169]:
test = test.iloc[:600,:]

In [171]:
txt = test['txt'].values

In [ ]:
E_train_ = t.texts_to_sequences(X_train_r)
E_test = t.texts_to_sequences(txt)

In [ ]:
padded_test = pad_sequences(E_test, maxlen=max_length, padding='post')

In [176]:
pred = RNN.predict(padded_test)>0.5

19/19 [==============================] - 0s 5ms/step


In [177]:
results = pd.DataFrame(pred, columns=['Predictions'])
results['Predicted'] = 0
results.loc[results['Predictions']==1, 'Predicted'] = 1
final_results = results['Predicted']

In [179]:
final_results.to_csv("../data/RNN_600_undersampled.csv", index=True)